<a href="https://colab.research.google.com/github/2001abdulrehman/FYP/blob/main/Lump_Eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRAINING**

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Extract the data from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/lump eye/lump Train.zip', 'r') as zip_ref:
    zip_ref.extractall('eye_images')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your data
train_dir = '/content/eye_images/Train'  # Update this path



# Create an image data generator
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Adjust rotation range to simulate different lash angles
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Adjust brightness to highlight lashes
    fill_mode='nearest'
)

train_generator = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust the image size as needed
    batch_size=32,
    class_mode='binary'  # Use 'categorical' for multiple classes
)

# Build the CNN model
model = keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3),
           data_format="channels_last"),  # Input layer with Conv2D
    MaxPooling2D(2, 2),  # Max-pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second Conv2D layer
    MaxPooling2D(2, 2),  # Second Max-pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Third Conv2D layer
    MaxPooling2D(2, 2),  # Third Max-pooling layer
    Flatten(),  # Flatten layer
    Dense(512, activation='relu'),  # Dense (fully connected) layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,  # You can adjust the number of epochs
    verbose=1
)

# Save the trained model
model.save('lump_detection_model.h5')


Found 949 images belonging to 2 classes.
Epoch 1/20
30/30 [==============================] - 64s 2s/step - loss: 0.8236 - accuracy: 0.5364
Epoch 2/20
30/30 [==============================] - 60s 2s/step - loss: 0.5943 - accuracy: 0.6702
Epoch 3/20
30/30 [==============================] - 59s 2s/step - loss: 0.4830 - accuracy: 0.7671
Epoch 4/20
30/30 [==============================] - 61s 2s/step - loss: 0.4147 - accuracy: 0.8230
Epoch 5/20
30/30 [==============================] - 62s 2s/step - loss: 0.3929 - accuracy: 0.8367
Epoch 6/20
30/30 [==============================] - 60s 2s/step - loss: 0.4749 - accuracy: 0.7893
Epoch 7/20
30/30 [==============================] - 64s 2s/step - loss: 0.4031 - accuracy: 0.8230
Epoch 8/20
30/30 [==============================] - 60s 2s/step - loss: 0.3320 - accuracy: 0.8588
Epoch 9/20
30/30 [==============================] - 65s 2s/step - loss: 0.2906 - accuracy: 0.8757
Epoch 10/20
30/30 [==============================] - 66s 2s/step - loss: 0.26

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf

# Load the pre-trained model in H5 format
model_path = '/content/lump_detection_model.h5'  # Replace with the path to your H5 model
model = tf.keras.models.load_model(model_path)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a .tflite file
tflite_path = 'lump_eye.tflite'  # Replace with the desired path for the TFLite model
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted and saved to {tflite_path}")

Model converted and saved to lump_eye.tflite


**TESTING**

In [ ]:
# Extract the testing data from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/lump eye/Lump Test.zip', 'r') as zip_ref:
    zip_ref.extractall('testing_data')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

# Load the TensorFlow Lite model
tflite_model_path = '/content/lump_eye.tflite'  # Replace with the path to your .tflite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define the path to the extracted test dataset directory
test_dir = '/content/testing_data'

# Initialize variables to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through the images in the test dataset
for subdir, _, files in os.walk(test_dir):
    for file in files:
        image_path = os.path.join(subdir, file)

        # Load and preprocess the image
        image = load_img(image_path, target_size=(150, 150))
        image = img_to_array(image) / 255.0  # Normalize the image
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        # Perform inference using the TFLite model
        interpreter.set_tensor(input_details[0]['index'], image)
        interpreter.invoke()
        tflite_predictions = interpreter.get_tensor(output_details[0]['index'])

        # Store true and predicted labels
        true_labels.append(1 if 'lump' in subdir.lower() else 0)
        predicted_labels.append(tflite_predictions[0][0])

# Convert the lists to numpy arrays for comparison
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate accuracy
threshold = 0.5  # Threshold for binary classification
predicted_classes = (predicted_labels >= threshold).astype('int32')
correct_predictions = np.sum(predicted_classes == true_labels)
accuracy = correct_predictions / len(true_labels)

print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


Accuracy on the test set: 50.75%


**RANDOM**

In [ ]:
import tensorflow as tf
from tensorflow.lite.python import interpreter as interpreter_wrapper
from PIL import Image
import os
import numpy as np  # Import NumPy
# Load the TFLite model
tflite_path = '/content/lump_eye.tflite'  # Replace with the path to your TFLite model
interpreter = interpreter_wrapper.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Define the path to the folder containing mixed lump and normal images
image_folder = '/content/drive/MyDrive/lump eye/test random'  # Replace with the path to your image folder

# Create empty lists to store lump and normal image filenames
lump_images = []
normal_images = []

# Iterate over the images in the folder and classify them
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)

        # Load the image
        image = Image.open(image_path)
        image = image.resize((150, 150))  # Resize to match the model's input size
        image = tf.convert_to_tensor(image, dtype=tf.float32) / 255.0  # Normalize the image
        image = image.numpy()
        image = image[np.newaxis, ...]  # Add a batch dimension

        # Set input tensor
        input_details = interpreter.get_input_details()
        interpreter.set_tensor(input_details[0]['index'], image)

        # Invoke the interpreter
        interpreter.invoke()

        # Get the output tensor
        output_details = interpreter.get_output_details()
        output = interpreter.get_tensor(output_details[0]['index'])

        # Classify the image
        is_lump = output[0][0] > 0.5  # Adjust the threshold as needed

        # Separate lump and normal images
        if is_lump:
            lump_images.append(filename)
        else:
            normal_images.append(filename)

# Display the lump and normal image filenames
print("Lump Images:")
for filename in lump_images:
    print(filename)

print("\nNormal Images:")
for filename in normal_images:
    print(filename)


Lump Images:
AA (9).jpg
AA (4).jpg
AA (7).jpg
AA (6).jpg
AA (8).jpg
AA (5).jpg
AA (3).jpg
AA (20).jpg
AA (14).jpg
AA (13).jpg
AA (1).jpg
AA (11).jpg
AA (15).jpg
AA (10).jpg
AA (16).jpg
AA (19).jpg
AA (2).jpg
AA (18).jpg
AA (17).jpg
AA (12).jpg

Normal Images:
BB (1).png
BB (9).png
BB (8).png
BB (5).png
BB (4).png
BB (6).png
BB (7).png
BB (20).png
BB (19).png
BB (3).png
BB (2).png
BB (18).png
BB (17).png
BB (16).png
BB (15).png
BB (13).png
BB (14).png
BB (12).png
BB (10).png
BB (11).png


In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np

# Load the TFLite model
tflite_path = '/content/drive/MyDrive/lump eye/lump_eye.tflite'  # Replace with the path to your TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Define a function to classify an image
def classify_image(image_path):
    # Load and preprocess the image
    image = Image.open(image_path)
    image = image.resize((150, 150))  # Resize to match the model's input size
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add a batch dimension

    # Set input tensor
    input_details = interpreter.get_input_details()
    interpreter.set_tensor(input_details[0]['index'], image)

    # Invoke the interpreter
    interpreter.invoke()

    # Get the output tensor
    output_details = interpreter.get_output_details()
    output = interpreter.get_tensor(output_details[0]['index'])

    # Classify the image
    is_lump = output[0][0] > 0.5  # Adjust the threshold as needed

    # Return the classification result
    return "Lump eye" if is_lump else "Normal eye"

# Define the path to the image you want to classify
image_to_classify = '/content/download.jfif'  # Replace with the path to your image

# Classify the image
result = classify_image(image_to_classify)
print(f"The image is classified as: {result}")


The image is classified as: Normal eye


**NEW TRAINING**

In [ ]:
# Extract the data from the zip file
with zipfile.ZipFile('/content/drive/MyDrive/lump eye/LUMP train 2.zip', 'r') as zip_ref:
    zip_ref.extractall('lump_images')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your data
train_dir = '/content/lump_images'  # Update this path



# Create an image data generator
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Adjust rotation range to simulate different lash angles
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Adjust brightness to highlight lashes
    fill_mode='nearest'
)

train_generator = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust the image size as needed
    batch_size=32,
    class_mode='binary'  # Use 'categorical' for multiple classes
)

# Build the CNN model
model = keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3),
           data_format="channels_last"),  # Input layer with Conv2D
    MaxPooling2D(2, 2),  # Max-pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second Conv2D layer
    MaxPooling2D(2, 2),  # Second Max-pooling layer
    Conv2D(128, (3, 3), activation='relu'),  # Third Conv2D layer
    MaxPooling2D(2, 2),  # Third Max-pooling layer
    Flatten(),  # Flatten layer
    Dense(512, activation='relu'),  # Dense (fully connected) layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,  # You can adjust the number of epochs
    verbose=1
)

# Save the trained model
model.save('lump_detection_model.h5')


Found 470 images belonging to 1 classes.
Epoch 1/20
15/15 [==============================] - 28s 2s/step - loss: 0.0484 - accuracy: 0.9319
Epoch 2/20
15/15 [==============================] - 35s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/20
15/15 [==============================] - 27s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/20
15/15 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/20
15/15 [==============================] - 27s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/20
15/15 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/20
15/15 [==============================] - 26s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/20
15/15 [==============================] - 28s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/20
15/15 [==============================] - 26s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/20
15/15 [==========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the pre-trained model in H5 format
model_path = '/content/lump_detection_model.h5'  # Replace with the path to your H5 model
model = tf.keras.models.load_model(model_path)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a .tflite file
tflite_path = 'lump_eye-2.tflite'  # Replace with the desired path for the TFLite model
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted and saved to {tflite_path}")

Model converted and saved to lump_eye-2.tflite


In [ ]:
import tensorflow as tf
from tensorflow.lite.python import interpreter as interpreter_wrapper
from PIL import Image
import numpy as np

def classify_single_image(image_path, tflite_model_path):
    # Load the TFLite model
    interpreter = interpreter_wrapper.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Load the image
    image = Image.open(image_path)
    image = image.resize((150, 150))  # Resize to match the model's input size
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add a batch dimension

    # Set input tensor
    input_details = interpreter.get_input_details()
    interpreter.set_tensor(input_details[0]['index'], image)

    # Invoke the interpreter
    interpreter.invoke()

    # Get the output tensor
    output_details = interpreter.get_output_details()
    output = interpreter.get_tensor(output_details[0]['index'])

    # Classify the image
    is_lump = output[0][0] > 0.5  # Adjust the threshold as needed

    # Return classification result
    return "Lump" if is_lump else "Normal"

# Replace with the path to your single image
image_to_classify = '/content/zain.jpg'

# Replace with the path to your TFLite model
model_path = '/content/lump_eye.tflite'

result = classify_single_image(image_to_classify, model_path)
print(f"The image is classified as: {result}")


The image is classified as: Normal
